<a href="https://colab.research.google.com/github/virtualspark/YCNG232-NLP_Fundamentals/blob/main/Reflexion_Week_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk.corpus import words
from nltk.metrics.distance import (
    edit_distance,
    jaccard_distance,
    )
from nltk.util import ngrams
nltk.download('words')
nltk.download('punkt')
import pandas
from nltk import sent_tokenize, word_tokenize

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Question 1

In [3]:
correct_spellings = words.words()
spellings_series = pandas.Series(correct_spellings)
spellings_series

0                 A
1                 a
2                aa
3               aal
4             aalii
            ...    
236731       yellow
236732          yes
236733    yesterday
236734          you
236735        young
Length: 236736, dtype: object

In [4]:
sentence = "his frined is also a greate companieon"

In [5]:
sent_tokenize = nltk.word_tokenize(sentence)

In [6]:
def jaccard_distance_method(sentence, ngram_number):
  all_outcome = []
  for word in sentence:
    temp = [(jaccard_distance(set(ngrams(word, ngram_number)), set(ngrams(w, ngram_number))), w) for w in correct_spellings if w[0]==word[0]]
    outcome = sorted(temp, key = lambda val:val[0])[0][1]
    all_outcome.append(outcome)
  return all_outcome

In [7]:
jaccard_distance_method(sent_tokenize, 1)

['his', 'finder', 'is', 'also', 'a', 'garetta', 'camptonite']

In [8]:
def edit_distance_method(sentence):
  all_outcome = []
  for word in sent_tokenize:
      temp = [(edit_distance(word, w),w) for w in correct_spellings if w[0]==word[0]]
      outcome = sorted(temp, key = lambda val:val[0])[0][1]
      all_outcome.append(outcome)
  return all_outcome

In [9]:
edit_distance_method(sent_tokenize)

['his', 'fried', 'is', 'also', 'a', 'grate', 'companion']

In [10]:
# pip install textblob

In [11]:
from textblob import Word
from textblob import TextBlob

In [12]:
def auto_corrector(input_sentence):
  sentence = TextBlob(input_sentence)
  result = sentence.correct()
  return print(result)

In [13]:
auto_corrector(sentence)

his friend is also a great companion


In [14]:
# Question 2

In [15]:
import nltk
import numpy as np
import random
import string

import bs4 as bs
import urllib.request
import re

In [16]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+') # remove punctuation

In [17]:
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Tennis')

In [18]:
raw_html = raw_html.read()

In [19]:
article_html = bs.BeautifulSoup(raw_html, 'lxml')
article_paragraphs = article_html.find_all('p')
article_text = ''

In [20]:
for para in article_paragraphs:
    article_text += para.text

In [21]:
article_text = article_text.lower()
article_text = re.sub(r'[^A-Za-z. ]', '', article_text)

In [22]:
article_text

'tennis is a racket sport that is played either individually against a single opponent singles or between two teams of two players each doubles. each player uses a tennis racket that is strung with cord to strike a hollow rubber ball covered with felt over or around a net and into the opponents court. the object of the game is to manoeuvre the ball in such a way that the opponent is not able to play a valid return. the player who is unable to return the ball validly will not gain a point while the opposite player will.tennis is an olympic sport and is played at all levels of society and at all ages. the sport can be played by anyone who can hold a racket including wheelchair users. the modern game of tennis originated in birmingham england in the late th century as lawn tennis. it had close connections both to various field lawn games such as croquet and bowls as well as to the older racket sport today called real tennis.the rules of modern tennis have changed little since the s. two e

In [23]:
# Auto Complete - Trigram - single word input

In [24]:
tokens = tokenizer.tokenize(article_text.lower()) # all in lower case 
find = nltk.TrigramCollocationFinder.from_words(tokens) # of course there is a Trigram one in NTLK
pmi = find.score_ngrams(nltk.TrigramAssocMeasures().pmi) # score with PMI

In [25]:
word = 'tennis'
for (pair, score) in pmi:
  (first, second, third) = pair
  if first == word:
    print(second, third, score)

ballwas racketted 19.482764622578035
writer steve 19.482764622578035
rating program 18.482764622578035
magazine selected 17.897802121856877
com polled 17.482764622578035
channel published 16.897802121856877
com panel 16.897802121856877
started using 16.482764622578035
association administers 16.312839621135723
rackets wilson 16.312839621135723
channel ranked 16.160836527690673
declined new 16.160836527690673
physique before 16.160836527690673
australia australian 15.675409700520431
after watching 15.575874026969517
federation iltf 15.482764622578035
tournament necessitates 15.395301781327696
federation itf 14.897802121856879
playing nations 14.897802121856879
strings available 14.897802121856879
most football 14.43837050321958
open semiopen 14.43837050321958
professionals atp 14.395301781327696
tournament took 14.395301781327696
almost all 14.312839621135723
an epitaph 14.312839621135723
stems from 14.312839621135723
withdrew from 14.312839621135723
often means 14.160836527690673
court

In [26]:
def autocomplete(word, scoring, k = 3):
  top = []
  bot = []
  for (pair, score) in scoring:
   (first, second, third) = pair
   if first == word and second != word and third != word: # no word repetitions, those would just loop
       if len(top) < k or score >= min(top, key = lambda o: o[1])[1]:
         top.append((second, score))
         top.append((third, score))
       if len(bot) < k or score <= max(bot, key = lambda o: o[1])[1]:
         bot.append((second, score))
         bot.append((third, score))
       if len(bot) > k:
         # ditch the HIGHEST score from bot(tom)
         bot.remove(max(bot, key = lambda o: o[1]))
       if len(top) > k:
         # ditch the LOWEST score from top
         top.remove(min(top, key = lambda o: o[1]))
  return (top, bot)

In [27]:
print(autocomplete('tennis', pmi, k = 3))

([('racketted', 19.482764622578035), ('writer', 19.482764622578035), ('steve', 19.482764622578035)], [('french', 10.978938884582284), ('open', 10.978938884582284), ('club', 10.84251968635569), ('on', 10.84251968635569), ('strings', 10.575874026969515), ('such', 10.575874026969515), ('followed', 10.45485862600815), ('by', 10.45485862600815), ('has', 10.14737426788411), ('been', 10.14737426788411), ('player', 10.105554092189482), ('ever', 10.105554092189482), ('player', 10.105554092189482), ('known', 10.105554092189482), ('according', 10.084020930639841), ('to', 10.084020930639841), ('tournament', 9.9358701626904), ('this', 9.9358701626904), ('and', 9.73289519518119), ('croquet', 9.73289519518119), ('whenever', 9.68024825745681), ('the', 9.68024825745681), ('game', 9.67540970052043), ('during', 9.67540970052043), ('fans', 9.668983431361), ('to', 9.668983431361), ('for', 9.668983431361), ('wta', 9.668983431361), ('when', 9.668983431361), ('an', 9.668983431361), ('tour', 9.649874608413292)

In [28]:
# Auto Complete - Quadgram - single word input

In [29]:
tokens = tokenizer.tokenize(article_text.lower()) # all in lower case 
find = nltk.QuadgramCollocationFinder.from_words(tokens) # of course there is a Quadgram one in NTLK
pmi = find.score_ngrams(nltk.QuadgramAssocMeasures().pmi) # score with PMI

In [30]:
word = 'tennis'
for (pair, score) in pmi:
  (first, second, third, fourth) = pair
  if first == word:
    print(second, third, fourth, score)

writer steve flink 32.81410147887452
rating program ntrp 31.814101478874516
com panel selected 28.644176477432204
rackets wilson head 28.05921397671105
after watching british 27.907210883265996
magazine selected martina 27.907210883265996
most football codes 27.76970735951606
ballwas racketted from 27.644176477432204
sphairistike this template 27.35466986023722
com polled its 27.11366176073342
channel ranked williams 26.492173383987154
open semiopen closed 26.184744858794907
federation itf conducts 25.907210883265996
federation itf defines 25.907210883265996
federation iltf now 25.814101478874516
declined new racket 25.68481846192955
scores from zero 25.644176477432204
often means having 25.492173383987154
balls must conform 25.17024528909979
tournament took place 25.141676136903023
have changed little 25.09985596120839
historian and journalist 24.64919455219883
outdoors and accordingly 24.64919455219883
rules were violated 24.421784056095756
started using this 24.35466986023722
club o

In [31]:
def autocomplete(word, scoring, k = 4):
  top = []
  bot = []
  for (pair, score) in scoring:
   (first, second, third, fourth) = pair
   if first == word and word not in (second, third, fourth): # no word repetitions, those would just loop
       if len(top) < k or score >= min(top, key = lambda o: o[1])[1]:
         top.append((second, score))
         top.append((third, score))
         top.append((fourth, score))
       if len(bot) < k or score <= max(bot, key = lambda o: o[1])[1]:
         bot.append((second, score))
         bot.append((third, score))
         bot.append((fourth, score))
       if len(bot) > k:
         # ditch the HIGHEST score from bot(tom)
         bot.remove(max(bot, key = lambda o: o[1]))
       if len(top) > k:
         # ditch the LOWEST score from top
         top.remove(min(top, key = lambda o: o[1]))
  return (top, bot)

In [32]:
print(autocomplete('tennis', pmi, k = 4))

([('writer', 32.81410147887452), ('steve', 32.81410147887452), ('flink', 32.81410147887452), ('ntrp', 31.814101478874516)], [('channel', 22.06926764137497), ('published', 22.06926764137497), ('a', 22.06926764137497), ('circuit', 22.06423205147767), ('and', 22.06423205147767), ('revenues', 22.06423205147767), ('hall', 21.97647354570311), ('of', 21.97647354570311), ('fame', 21.97647354570311), ('association', 21.83039528621517), ('to', 21.83039528621517), ('teach', 21.83039528621517), ('followed', 21.78619548230463), ('by', 21.78619548230463), ('rafael', 21.78619548230463), ('tournament', 21.682244518265723), ('this', 21.682244518265723), ('draw', 21.682244518265723), ('professionals', 21.639175796373838), ('atp', 21.639175796373838), ('or', 21.639175796373838), ('must', 21.469805570958698), ('use', 21.469805570958698), ('balls', 21.469805570958698), ('physique', 21.327266457311467), ('before', 21.327266457311467), ('and', 21.327266457311467), ('competitions', 21.322248382544842), ('thes

In [33]:
# Auto Complete - Bigram - 2 words input

In [34]:
ngrams = {}
words = 2

words_tokens = nltk.word_tokenize(article_text)
for i in range(len(words_tokens)-words):
    seq = ' '.join(words_tokens[i:i+words])
    # print(seq)
    if  seq not in ngrams.keys():
        ngrams[seq] = []
    ngrams[seq].append(words_tokens[i+words])

In [35]:
words_tokens[0:words]

['tennis', 'is']

In [36]:
curr_sequence = ' '.join(['tennis', 'is'])
output = curr_sequence

In [37]:
curr_sequence

'tennis is'

In [38]:
for i in range(50):
    if curr_sequence not in ngrams.keys():
        break
    possible_words = ngrams[curr_sequence]
    next_word = possible_words[random.randrange(len(possible_words))]
    output += ' ' + next_word
    seq_words = nltk.word_tokenize(output)
    curr_sequence = ' '.join(seq_words[len(seq_words)-words:len(seq_words)])

print(output)

tennis is the most natural tennis mind with the wellknown phrase game set match followed by the first player to win the game . the terminology is extended to sets set point matches match point and even championships championship point . this stance allows for effective torso rotation which can result in


In [39]:
# Auto Complete - Trigram - 3 words input

In [40]:
ngrams = {}
words = 3

words_tokens = nltk.word_tokenize(article_text)
for i in range(len(words_tokens)-words):
    seq = ' '.join(words_tokens[i:i+words])
    # print(seq)
    if  seq not in ngrams.keys():
        ngrams[seq] = []
    ngrams[seq].append(words_tokens[i+words])

In [41]:
curr_sequence = ' '.join(['tennis', 'is', 'a'])
output = curr_sequence

In [42]:
for i in range(50):
    if curr_sequence not in ngrams.keys():
        break
    possible_words = ngrams[curr_sequence]
    next_word = possible_words[random.randrange(len(possible_words))]
    output += ' ' + next_word
    seq_words = nltk.word_tokenize(output)
    curr_sequence = ' '.join(seq_words[len(seq_words)-words:len(seq_words)])

print(output)

tennis is a shot returned to the opponent in midair before the ball is spinning counterclockwise it will curve right from the hitters point of view and curve left if spinning clockwise.some servers are content to use the serve simply to initiate the point however advanced players often try to hit a winning


In [43]:
# Auto Complete - Quadgram - 4 words input

In [44]:
ngrams = {}
words = 4

words_tokens = nltk.word_tokenize(article_text)
for i in range(len(words_tokens)-words):
    seq = ' '.join(words_tokens[i:i+words])
    # print(seq)
    if  seq not in ngrams.keys():
        ngrams[seq] = []
    ngrams[seq].append(words_tokens[i+words])

In [45]:
curr_sequence = ' '.join(['tennis', 'is', 'a', 'racket'])
output = curr_sequence

In [46]:
for i in range(50):
    if curr_sequence not in ngrams.keys():
        break
    possible_words = ngrams[curr_sequence]
    next_word = possible_words[random.randrange(len(possible_words))]
    output += ' ' + next_word
    seq_words = nltk.word_tokenize(output)
    curr_sequence = ' '.join(seq_words[len(seq_words)-words:len(seq_words)])

print(output)

tennis is a racket sport that is played either individually against a single opponent singles or between two teams of two players each doubles . each player uses a tennis racket that is strung with cord to strike a hollow rubber ball covered with felt over or around a net and into the opponents


In [47]:
# Question 3

In [48]:
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [49]:
from nltk.corpus import names

In [50]:
name_corpus = names.words()
name_corpus_series = pandas.Series(name_corpus)
name_corpus_series

0       Abagael
1       Abagail
2          Abbe
3         Abbey
4          Abbi
         ...   
7939       Zeus
7940      Zippy
7941     Zollie
7942      Zolly
7943      Zorro
Length: 7944, dtype: object

In [51]:
name_corpus_series = name_corpus_series.str.lower()

In [52]:
def autocomplete(word, scoring, k = 4):
  top = []
  bot = []
  for (pair, score) in scoring:
   (first, second, third, fourth) = pair
   if first == word and word not in (second, third, fourth) and not name_corpus_series.str.contains(first).any() \
   and not name_corpus_series.str.contains(second).any() \
   and not name_corpus_series.str.contains(third).any() \
   and not name_corpus_series.str.contains(fourth).any(): # no word repetitions, those would just loop
       if len(top) < k or score >= min(top, key = lambda o: o[1])[1]:
         top.append((second, score))
         top.append((third, score))
         top.append((fourth, score))
       if len(bot) < k or score <= max(bot, key = lambda o: o[1])[1]:
         bot.append((second, score))
         bot.append((third, score))
         bot.append((fourth, score))
       if len(bot) > k:
         # ditch the HIGHEST score from bot(tom)
         bot.remove(max(bot, key = lambda o: o[1]))
       if len(top) > k:
         # ditch the LOWEST score from top
         top.remove(min(top, key = lambda o: o[1]))
  return (top, bot)

In [53]:
print(autocomplete('tennis', pmi, k = 4))

([('rating', 31.814101478874516), ('program', 31.814101478874516), ('ntrp', 31.814101478874516), ('british', 27.907210883265996)], [('conducts', 25.907210883265996), ('federation', 25.907210883265996), ('itf', 25.907210883265996), ('defines', 25.907210883265996), ('federation', 25.814101478874516), ('iltf', 25.814101478874516), ('now', 25.814101478874516), ('scores', 25.644176477432204), ('from', 25.644176477432204), ('zero', 25.644176477432204), ('often', 25.492173383987154), ('means', 25.492173383987154), ('having', 25.492173383987154), ('tournament', 25.141676136903023), ('took', 25.141676136903023), ('place', 25.141676136903023), ('started', 24.35466986023722), ('using', 24.35466986023722), ('this', 24.35466986023722), ('australia', 23.96235243745846), ('australian', 23.96235243745846), ('open', 23.96235243745846), ('together', 23.35466986023722), ('these', 23.35466986023722), ('four', 23.35466986023722), ('association', 22.862816763907546), ('wta', 22.862816763907546), ('ranking',